# Create Periodic Project Reports in Smartsheet

This should be run once a week or so, to post project updates, for project situational awareness: how many hours are left, how many are worked.

In [ ]:
%load_ext nb_black

import os
import json
import logging
from datetime import datetime, timedelta
from typing import Dict, List

import pandas as pd
import numpy as np
import prefect
from box import Box

import smartsheet

# uses the pretty okay SDK here: https://github.com/ProdPerfect/monday
from monday import MondayClient
from mondaydotcom_utils.formatted_value import FormattedValue, get_col_defs
from mondaydotcom_utils.time_block import TimeBlock
from mondaydotcom_utils.utilities import (
    breakout_record,
    get_items_by_board,
    validate_task_record,
)
from prefect import Flow, Parameter, task, unmapped
from prefect.executors import LocalDaskExecutor, LocalExecutor

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
TASKS_BOARD_ID = "1883170887"
AGREEMENTS_BOARD_ID = "1882423671"
PROJECTS_BOARD_ID = "1882404316"
ACCOUNTS_BOARD_ID = "1882424009"

PROJECT_TASK_TIME_BOARD_ID = "2398200403"

unposted_sheet_id = 4818113414883204
posted_tasks_id = 3567675495475076

# don't set this here for development work... use the secrets-<environment>.yaml files instead.
MONDAY_KEY = ""
SMARTSHEET_KEY = ""
environment = "dev"

In [ ]:
if not MONDAY_KEY:
    # key hasn't been passed as a papermill parameter... get it from a file?
    secrets = Box.from_yaml(filename=f"secrets-{environment}.yaml")
    MONDAY_KEY = secrets.apps.monday.API_KEY

if not SMARTSHEET_KEY:
    # key hasn't been passed as a papermill parameter... get it from a file?
    secrets = Box.from_yaml(filename=f"secrets-{environment}.yaml")
    SMARTSHEET_KEY = secrets.apps.smartsheet.API_KEY

In [ ]:
# connect monday client
conn = MondayClient(MONDAY_KEY)

In [ ]:
# connect smartsheet client
ss_client = smartsheet.Smartsheet(SMARTSHEET_KEY)
ss_client.errors_as_exceptions(True)

In [ ]:
users = conn.users.fetch_users()["data"]["users"]
users_df = pd.DataFrame(users).set_index("id")
users_df.head()

In [ ]:
def breakdown_status(x):
    # use this to break down the status columns
    # TODO move this to mondaydotcom-utils in the formatters

    my_list = []
    json1 = json.loads(x)

    if json1.get("text"):
        my_list.append(json1["text"])
    if json1.get("changed_at"):
        my_list.append(json1["changed_at"])

    return ";".join(my_list)

In [ ]:
def get_status_text(x):
    if x:
        json1 = json.loads(x)

        if json1.get("text"):
            return json1["text"]
    return "None"

In [ ]:
accounts_df = get_items_by_board(conn, ACCOUNTS_BOARD_ID)

accounts_df.rename(
    columns={"monday_id": "account_id", "Title": "Client Name"},
    inplace=True,
)

accounts_df.drop(
    columns=[
        "Contacts",
        "Item ID",
        "Subitems",
        "Notes",
        "Customer Projects",
        "Agreements",
        "Type",
    ],
    inplace=True,
    errors="ignore",
)

# convert the yes-no to True-False
accounts_df["No Bill"] = accounts_df["No Bill"].apply(
    lambda x: bool(json.loads(x)["checked"]) if x else False
)

accounts_df

In [ ]:
projects_df = get_items_by_board(conn, PROJECTS_BOARD_ID)

projects_df.rename(
    columns={
        "monday_id": "project_id",
    },
    inplace=True,
)

projects_df.drop(
    columns=[
        "Repo Description (mirror)",
        "Project Tasks",
        "Subitems",
        "Etimated Time (Hours) (mirror)",
        "Total Task Time (Hours) (mirror)",
        "Project Contacts",
        "SET Resource",
        "Timeline",
        "Customer Source",
        "Tasks Status (mirror)",
        "Dependency",
        "Date Added",
        "Timeline Days",
        "Item ID",
        "Project Health",
    ],
    inplace=True,
    errors="ignore",
)

projects_df["Project Lifecycle"] = projects_df["Project Lifecycle"].apply(
    get_status_text
)

projects_df = projects_df.explode(["Account"], ignore_index=True)
projects_df

In [ ]:
# add the account to the projects
projects_df = pd.merge(
    projects_df, accounts_df, how="left", left_on="Account", right_on="account_id"
).drop(columns=["account_id"])
projects_df

In [ ]:
def breakout_time_sessions(row):
    """
    Break down the Monday.com time structure into something simpler for us.

    This is used with a DataFrame.apply()
    """

    mct = TimeBlock()
    mct.parse(row["Actual Time"])
    return mct.total_duration_hours, mct.time_records

In [ ]:
# only getting not posted tasks
tasks_df = get_items_by_board(conn, TASKS_BOARD_ID)

# Only include Ready tasks
tasks_df = tasks_df.loc[
    ~tasks_df["Integration Message"].str.startswith("Posted", na=False)
]

tasks_df.rename(
    columns={
        "monday_id": "task_id",
    },
    inplace=True,
)

# break the time sessions out
tasks_df[["Total Duration Hours", "Time Sessions"]] = tasks_df.apply(
    breakout_time_sessions, axis=1, result_type="expand"
)

tasks_df["Hours"] = tasks_df["Actual Hours"].fillna(0) + tasks_df[
    "Total Duration Hours"
].fillna(0)
tasks_df["Estimated Hours"] = tasks_df["Timeline Days"].fillna(0) * 8

tasks_df["Status"] = tasks_df["Status"].apply(get_status_text)

tasks_df.drop(
    columns=[
        "Subtasks",
        "Timeline Hours (Estimated) (formula)",
        "Total Actual Hours (formula)",
        "Customer Repos",
        "Project Lifecycle (mirror)",
        "task_id",
        "Dependencies",
        "Issue URL",
        "Pull Request URL",
        "Owner",
        "Actual Hours",
        "Actual Time",
        "Total Duration Hours",
        "Time Sessions",
        "Integration Message",
        "Timeline",
        "Timeline Days",
        "Date Added",
        "Actual Time",
        "Date Completed",
        "Notes",
        "Date Added",
    ],
    inplace=True,
    errors="ignore",
)

# projects should be limited to just one, so this will bring it out of the list
tasks_df = tasks_df.explode(["Customer Project"], ignore_index=True)
tasks_df.head()

Finally merge the tasks and projects together for a final task list.

In [ ]:
df = pd.merge(
    tasks_df,
    projects_df,
    how="left",
    left_on="Customer Project",
    right_on="project_id",
)

# We only the wants those we bill for
df = df.loc[~df["No Bill"]]

df.rename(
    columns={
        "Title_x": "Title",
        "Title_y": "Project Title",
        "Hours": "Task Hours",
    },
    inplace=True,
)

df.drop(
    columns=[
        "monday_id",
        "task_end_year",
        "task_end_month",
        "Account",
        "No Bill",
    ],
    inplace=True,
    errors="ignore",
)

df.head()

In [ ]:
# Collect tasks that are complete, so estimates no longer matter.
report_done_df = (
    df.loc[df.Status == "Done"]
    .groupby(["Client Name", "Project Title"])
    .agg(
        {
            "Estimated Hours": "sum",
            "Task Hours": "sum",
            "Grant Number": "first",
            "Notes": "first",
            "Project Lifecycle": "first",
            "project_id": "first",
        }
    )
    .reset_index()
)

report_done_df["Estimated Hours"] = 0
report_done_df["Task Status"] = "Done"
report_done_df

In [ ]:
# collect tasks that are incomplete, subtracting the done time from the estimate
report_undone_df = (
    df.loc[df.Status != "Done"]
    .groupby(["Client Name", "Project Title"])
    .agg(
        {
            "Estimated Hours": "sum",
            "Task Hours": "sum",
            "Grant Number": "first",
            "Notes": "first",
            "Project Lifecycle": "first",
            "project_id": "first",
        }
    )
    .reset_index()
)
report_undone_df["Estimated Hours"] = (
    report_undone_df["Estimated Hours"] - report_undone_df["Task Hours"]
)
report_undone_df["Task Status"] = "Not Done"
report_undone_df

In [ ]:
# shuffle the done and undone together
report_df = (
    pd.concat([report_done_df, report_undone_df])
    .groupby(["Client Name", "Project Title"])
    .agg(
        {
            "Estimated Hours": "sum",
            "Task Hours": "sum",
            "project_id": "first",
            "Grant Number": "first",
            "Notes": "first",
            "Project Lifecycle": "first",
            # "Task Status": lambda x: ";".join(list(x)),
        }
    )
    .reset_index()
)
report_df

## Add the records to Smartsheet

In [ ]:
unposted_sheet = ss_client.Sheets.get_sheet(unposted_sheet_id)

In [ ]:
# break down the cell IDs into a quick lookup box
cell_ids = {}
for column in unposted_sheet.columns:
    my_column = column.to_dict()
    cell_ids[my_column["title"]] = my_column["id"]
cell_ids

In [ ]:
right_now = datetime.now().strftime("%Y-%m-%d")

rows = []
for k, v in report_df.to_dict("index").items():

    row = ss_client.models.row.Row()

    row.cells.append(
        {"column_id": cell_ids["Account/Client"], "value": v["Client Name"]}
    )
    if v.get("Grant Number"):
        row.cells.append(
            {"column_id": cell_ids["Grant Proposal #"], "value": v["Grant Number"]}
        )
    row.cells.append(
        {"column_id": cell_ids["Project Title"], "value": v["Project Title"]}
    )
    row.cells.append({"column_id": cell_ids["Reported Date"], "value": right_now})
    row.cells.append(
        {
            "column_id": cell_ids["Estimated Hours Remaining"],
            "value": v["Estimated Hours"],
        }
    )
    row.cells.append(
        {"column_id": cell_ids["Unposted Task Hours"], "value": v["Task Hours"]}
    )
    row.cells.append(
        {"column_id": cell_ids["Project Lifecycle"], "value": v["Project Lifecycle"]}
    )

    row.to_bottom = True
    rows.append(row)


result = ss_client.Sheets.add_rows(unposted_sheet_id, rows)

## Delete all estimates from posted

In [ ]:
posted_sheet = ss_client.Sheets.get_sheet(posted_tasks_id)

In [ ]:
# break down the cell IDs into a quick lookup box
posted_cell_ids = {}
for column in posted_sheet.columns:
    my_column = column.to_dict()
    posted_cell_ids[my_column["title"]] = my_column["id"]
posted_cell_ids

In [ ]:
# filter for estimates only
result = ss_client.Sheets.get_sheet(posted_tasks_id, filter_id=5850658663360388)
rows_to_delete = [x["id"] for x in result.to_dict()["rows"] if not x["filteredOut"]]

In [ ]:
result = ss_client.Sheets.delete_rows(posted_tasks_id, rows_to_delete)

## Add non-posted records FYI

In [ ]:
# add all unposted tasks hours as estimated only
rows = []
for k, v in report_df.to_dict("index").items():

    row = ss_client.models.row.Row()

    row.cells.append(
        {"column_id": posted_cell_ids["Account/Client"], "value": v["Client Name"]}
    )
    if v.get("Grant Number"):
        row.cells.append(
            {
                "column_id": posted_cell_ids["Grant Proposal #"],
                "value": v["Grant Number"],
            }
        )
    row.cells.append(
        {"column_id": posted_cell_ids["Notes"], "value": f"** Not posted **"}
    )
    row.cells.append(
        {"column_id": posted_cell_ids["Project Title"], "value": v["Project Title"]}
    )
    row.cells.append(
        {"column_id": posted_cell_ids["Month-end Date"], "value": right_now}
    )
    row.cells.append(
        {"column_id": posted_cell_ids["Completed Hours"], "value": v["Task Hours"]}
    )
    row.cells.append({"column_id": posted_cell_ids["Estimated Only"], "Value": True})

    row.to_bottom = True
    rows.append(row)


result = ss_client.Sheets.add_rows(posted_tasks_id, rows)